In [1]:
# imports
from PIL import Image
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [2]:
# load dataset
numbers = [2, 3, 4]
pics = 200
image_arrays = []
for i in numbers:
    for j in range(pics):
        file_path = os.path.join('dataset', str(i), f'{i}_{j}.png')
        if os.path.exists(file_path):
            image = Image.open(file_path)
            image_array = np.array(image)
            image_array_red_channel= image_array[:, :, 0]
            image_array_red_channel = image_array_red_channel.flatten()
            image_arrays.append(image_array_red_channel)

number = []
for i in numbers:
    for j in range(pics):
        number.append(i)

df = pd.DataFrame(image_arrays)
df=df/255
df['number'] = number


In [48]:
# train and test split
df_random = df.sample(frac=1, random_state=22)
split = int(0.7*len(df_random))
df_train = df_random.iloc[:split]
df_test = df_random.iloc[split:]

x_train = df_train.drop('number', axis=1).values
y_train = df_train['number'].values
y_train_encoded = to_categorical(y_train-2)
x_test = df_test.drop('number', axis=1).values
y_test = df_test['number'].values
y_test_encoded = to_categorical(y_test-2)


In [64]:
# model
model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation='relu'))  # Input layer
model.add(Dense(16, activation='relu'))  # Hidden layer
model.add(Dense(3, activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

In [65]:
early_stopping = EarlyStopping(monitor='val_loss',patience=15)
# Train the model
model.fit(x_train, y_train_encoded, validation_data=(x_test, y_test_encoded), epochs=200, batch_size=12, callbacks=[early_stopping])

Epoch 1/200


35/35 [==============================] - 1s 6ms/step - loss: 1.0111 - accuracy: 0.5071 - precision: 0.5867 - recall: 0.2095 - val_loss: 0.8378 - val_accuracy: 0.6611 - val_precision: 0.7545 - val_recall: 0.4611
Epoch 2/200
35/35 [==============================] - 0s 2ms/step - loss: 0.6965 - accuracy: 0.7452 - precision: 0.8449 - recall: 0.6357 - val_loss: 0.6482 - val_accuracy: 0.7556 - val_precision: 0.8403 - val_recall: 0.6722
Epoch 3/200
35/35 [==============================] - 0s 2ms/step - loss: 0.5410 - accuracy: 0.8000 - precision: 0.8462 - recall: 0.7595 - val_loss: 0.5650 - val_accuracy: 0.8167 - val_precision: 0.8500 - val_recall: 0.7556
Epoch 4/200
35/35 [==============================] - 0s 2ms/step - loss: 0.4888 - accuracy: 0.8381 - precision: 0.8692 - recall: 0.8071 - val_loss: 0.5617 - val_accuracy: 0.8444 - val_precision: 0.8488 - val_recall: 0.8111
Epoch 5/200
35/35 [==============================] - 0s 2ms/step - loss: 0.4043 - accuracy: 0.8762 - precision: 0.8995 -

In [66]:
# Evaluate the model
loss, accuracy, precision, recall = model.evaluate(x_test, y_test_encoded)
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('Loss: %.2f' % loss)

6/6 [==============================] - 0s 1ms/step - loss: 0.3797 - accuracy: 0.9222 - precision: 0.9222 - recall: 0.9222
Accuracy: 92.22
Precision: 92.22
Recall: 92.22
Loss: 0.38
